<a href="https://colab.research.google.com/github/JorgeAnsotegui/TFM/blob/main/Scripts%20de%20detecci%C3%B3n%20de%20Video/VideoDetectionYOLOv8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Conectar Colab a Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Instalar las dependencias necesarias
!pip install ultralytics opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 873.6/873.6 kB 45.5 MB/s eta 0:00:00


**Para descomprimir la carpeta de los modelos**

In [ ]:
# # Rutas de los archivos zip en Google Drive
# ruta_zip = '/content/drive/MyDrive/TFM/models/modelos.zip'

# # Rutas de las carpetas destino
# ruta_destino = '/content'

In [ ]:
# import os
# import zipfile

# # Crear la carpeta destino si no existen
# os.makedirs(ruta_destino, exist_ok=True)

# # Descomprimir los archivos zip
# with zipfile.ZipFile(ruta_zip, 'r') as zip_ref:
#     zip_ref.extractall(ruta_destino)

# print(f'Archivos descomprimidos en {ruta_destino}')

Archivos descomprimidos en /content


In [18]:
import cv2
import time
from ultralytics import YOLO
import os
import numpy as np

# Definir la ruta del modelo
# model_path = '/content/drive/MyDrive/TFM/models/modelos/YoloV8_n_fix.pt'
# model_path = '/content/drive/MyDrive/TFM/models/modelos/YoloV8_n_pre.pt'
# model_path = '/content/drive/MyDrive/TFM/models/modelos/YoloV8_n_all.pt'
# model_path = '/content/drive/MyDrive/TFM/models/modelos/YoloV8_s_fix.pt'
model_path = '/content/drive/MyDrive/TFM/models/modelos/YoloV8_s_pre.pt'
# model_path = '/content/drive/MyDrive/TFM/models/modelos/YoloV8_s_all.pt'


# Cargar el modelo YOLOv8 para segmentación
model = YOLO(model_path)

# Configuración
initial_frames_to_test = 10  # Número de frames iniciales para medir el tiempo de procesamiento

# Obtener el nombre del modelo sin la extensión
model_name = os.path.splitext(os.path.basename(model_path))[0]

# Definir la ruta de salida basada en el nombre del modelo
output_path = f"{model_name}.mp4"

video_path = "/content/drive/MyDrive/TFM/Cáncer Válvula Ileocecal 0:25 seg _1:05 .mp4"

# Función para medir el tiempo de procesamiento y determinar la frecuencia óptima
def determine_processing_interval(video_path):
    cap = cv2.VideoCapture(video_path)

    # Obtener la información del video
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = frame_count / fps

    # Medir el tiempo de procesamiento de los primeros `initial_frames_to_test` frames
    start_time = time.time()
    for _ in range(initial_frames_to_test):
        ret, frame = cap.read()
        if not ret:
            break
        # Procesar el frame con el modelo
        results = model(frame)
    processing_time = time.time() - start_time

    # Calcular el tiempo promedio por frame
    avg_processing_time_per_frame = processing_time / initial_frames_to_test

    # Calcular el intervalo de frames basado en el tiempo disponible
    max_processing_time_per_frame = 1 / fps  # Tiempo disponible por frame
    optimal_processing_interval = max(1, int(round(avg_processing_time_per_frame / max_processing_time_per_frame)))

    cap.release()

    return optimal_processing_interval

# Función para redimensionar la máscara al tamaño del frame
def resize_mask(mask, target_shape):
    return cv2.resize(mask.astype(float), (target_shape[1], target_shape[0]), interpolation=cv2.INTER_LINEAR) > 0.5

# Función para procesar video y guardar el resultado
def process_video(video_path, output_path, process_every_n_frames):
    cap = cv2.VideoCapture(video_path)

    # Obtener la información del video original
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Crear el objeto de escritura de video
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    frame_count = 0
    last_masks = None
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Procesar el frame si es necesario
        if frame_count % process_every_n_frames == 0:
            # Realizar la detección
            results = model(frame)

            # Obtener las máscaras del resultado
            if results and len(results) > 0:
                if hasattr(results[0], 'masks') and results[0].masks is not None:
                    last_masks = results[0].masks.data.cpu().numpy()
                else:
                    last_masks = None
            else:
                last_masks = None

            # Dibujar los resultados en el frame
            annotated_frame = results[0].plot() if results and len(results) > 0 else frame.copy()
        else:
            # Si no procesamos este frame, usamos el frame original y dibujamos las últimas máscaras
            annotated_frame = frame.copy()
            if last_masks is not None:
                for mask in last_masks:
                    resized_mask = resize_mask(mask[0], frame.shape[:2])
                    color = np.random.randint(0, 255, size=(3,))
                    colored_mask = np.zeros_like(annotated_frame)
                    colored_mask[resized_mask] = color
                    annotated_frame = cv2.addWeighted(annotated_frame, 1, colored_mask, 0.5, 0)

        # Escribir el frame con detecciones en el archivo de video
        out.write(annotated_frame)

        frame_count += 1

    cap.release()
    out.release()

# Determinar el intervalo óptimo de procesamiento
optimal_interval = determine_processing_interval(video_path)

print(f"Intervalo óptimo de procesamiento: cada {optimal_interval} frames")

# Procesar el video con el intervalo óptimo
process_video(video_path, output_path, optimal_interval)

print(f"El video procesado se guardó en: {output_path}")


0: 768x1024 (no detections), 77.0ms
Speed: 5.6ms preprocess, 77.0ms inference, 0.7ms postprocess per image at shape (1, 3, 768, 1024)

0: 768x1024 (no detections), 34.5ms
Speed: 6.0ms preprocess, 34.5ms inference, 0.6ms postprocess per image at shape (1, 3, 768, 1024)

0: 768x1024 (no detections), 33.9ms
Speed: 6.0ms preprocess, 33.9ms inference, 0.6ms postprocess per image at shape (1, 3, 768, 1024)

0: 768x1024 (no detections), 27.8ms
Speed: 6.3ms preprocess, 27.8ms inference, 0.6ms postprocess per image at shape (1, 3, 768, 1024)

0: 768x1024 (no detections), 27.7ms
Speed: 5.8ms preprocess, 27.7ms inference, 0.6ms postprocess per image at shape (1, 3, 768, 1024)

0: 768x1024 (no detections), 27.7ms
Speed: 6.2ms preprocess, 27.7ms inference, 0.6ms postprocess per image at shape (1, 3, 768, 1024)

0: 768x1024 (no detections), 27.8ms
Speed: 6.0ms preprocess, 27.8ms inference, 0.6ms postprocess per image at shape (1, 3, 768, 1024)

0: 768x1024 (no detections), 24.4ms
Speed: 6.0ms prepr

In [19]:
import shutil
import os

# Archivo de origen
src_file = output_path  # Asegúrate de que esto sea el archivo, no un directorio

# Directorio de destino
dest_dir = "/content/drive/MyDrive/TFM/Videos"

def copy_file(src, dest_dir):
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)

    dest_file = os.path.join(dest_dir, os.path.basename(src))
    shutil.copy2(src, dest_file)

# Copiar el archivo de origen al directorio de destino
copy_file(src_file, dest_dir)

print(f"Archivo copiado de {src_file} a {dest_dir}")


Archivo copiado de YoloV8_s_pre.mp4 a /content/drive/MyDrive/TFM/Videos
